In [34]:
from datasets import load_metric, load_from_disk
from transformers import (
    RobertaTokenizer,
    T5ForConditionalGeneration,
    DataCollatorForSeq2Seq,
)
import torch


In [50]:
torch.set_num_threads(16)

In [2]:
tokenizer = RobertaTokenizer.from_pretrained("Salesforce/codet5-small")
model = T5ForConditionalGeneration.from_pretrained("/data/nicolasmaier/model/codet5-merged-1/checkpoint-9000")
model.eval()

print("ok")


ok


In [13]:
dataset_test = load_from_disk("/data/nicolasmaier/dataset/hf_merged_test_1")
print("test:", dataset_test)


test: Dataset({
    features: ['code', 'input_ids', 'attention_mask', 'labels'],
    num_rows: 6000
})


In [52]:
def compute_predictions(examples):
    print("go")
    orig_input = [tokenizer.decode(x[1:-1]) for x in examples["input_ids"]]
    print("ok1")
    orig_output = [tokenizer.decode(x[1:-1]) for x in examples["labels"]]
    print("ok")

    model_input = tokenizer(
        orig_input, padding=True, truncation=True, return_tensors="pt"
    )
    print("ok2")

    outputs = model.generate(
        model_input.input_ids,
        attention_mask=model_input.attention_mask,
        num_beams=10,
        max_length=510,
        # do_sample=True,
        temperature=0.3,
        top_k=50,
        top_p=0.8,
    )
    print("ok3")

    return {"prediction": outputs}


xyz = dataset_test.select(range(100))
print(xyz)
dataset_results = dataset_test.select(range(4)).map(
    compute_predictions, batched=True, batch_size=2, num_proc=2
)


Dataset({
    features: ['code', 'input_ids', 'attention_mask', 'labels'],
    num_rows: 100
})
   

#0:   0%|          | 0/1 [00:00<?, ?ba/s]

go
ok1
ok
ok2
 

#1:   0%|          | 0/1 [00:00<?, ?ba/s]

go
ok1
ok
ok2


KeyboardInterrupt: 

In [24]:
print(compute_predictions(dataset_test.select(range(10))))

TypeError: int() argument must be a string, a bytes-like object or a number, not 'list'

In [10]:
print(tokenizer.decode(dataset_test.shuffle(21)[2]["input_ids"]))
print(tokenizer.decode(dataset_test.shuffle(21)[2]["labels"]))

Loading cached shuffled indices for dataset at /data/nicolasmaier/dataset/hf_merged_test_1/cache-3da988711144e5ae.arrow


<s>java to sequence
private PointFeature nextFilteredDataPoint() {
        while (origIter.hasNext()) {
            PointFeature pointFeat = origIter.next();
            if (filter.filter(pointFeat)) {
                return pointFeat;
            }
        }

        return null;
    }</s>
<s>{"title": "nextFilteredDataPoint()", "sequence": [{"type": "methodInvocation", "to": ["origIter"], "method": "hasNext()"}, {"type": "blocks", "name": "while", "blocks": [{"guard": "origIter.hasNext()", "contents": [{"type": "methodInvocation", "to": ["origIter"], "method": "next()"}, {"type": "scopedVariable", "name": "pointFeat"}, {"type": "methodInvocation", "to": ["filter"], "method": "filter(pointFeat)"}, {"type": "blocks", "name": "if", "blocks": [{"guard": "filter.filter(pointFeat)", "contents": [{"type": "controlFlow", "name": "return", "value": "pointFeat"}]}]}]}]}, {"type": "controlFlow", "name": "return", "value": "null"}]}</s>


In [ ]:
metric = load_metric("bleu")
print(metric_bleu.inputs_description)
